In [188]:
from nltk.corpus import stopwords
import codecs
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize, sent_tokenize
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold
from collections import defaultdict
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.svm import SVC
import time

In [189]:
def read_it(item):
    f=codecs.open(item ,'r')
    f=f.read()
    f=f.replace("\n", " ")
    sent_tokens = sent_tokenize(f)
    tokens = [sent for sent in map(word_tokenize, sent_tokens)]
    list(enumerate(tokens))
    others = '“,”,’,—'
    stopwords_ = set(stopwords.words('english'))
    tokens_lower = [[word.lower() for word in sent] for sent in tokens]
    punctuation_ = set(string.punctuation)
    clean_token= [[word.replace(str(punctuation_),'') for word in sent] for sent in tokens_lower]

    def filter_tokens(sent):
        return([w for w in sent if not w in stopwords_ and not w in punctuation_ and not w in others])

    tokens_filtered = list(map(filter_tokens, clean_token))



    return tokens_filtered

In [190]:
def chunk(x, y = 30):
    l=len(x)
    v=l//y
    remainder=y
    all_series = []
    for loops in range(v+1):
        new_elem = []
        if loops > (v - 1):
            remainder = l%y
        if remainder > 0:
            for i in range(remainder):
                temp = (y * (loops)) + i
                new_elem.extend(x[temp])
            all_series.append(new_elem)
    return pd.Series(all_series)

In [191]:
def make_it_df(item,title,author,year):
    df = pd.DataFrame()
    df['txt']= chunk(read_it(item))
    df['title']= title
    df['author'] = author
    df['year'] = year
    #df['genre'] = genre
    return df

In [192]:
bell = make_it_df("/Users/andrewargaez/Author_Classifier/WBT.txt",'For Whom the Bell Tolls','Ernest Hemmingway','1940')
fta = make_it_df("/Users/andrewargaez/Author_Classifier/FTA.txt", "A Farewell to Arms", "Ernest Hemingway",'1929')

kar = make_it_df("/Users/andrewargaez/Author_Classifier/karmaz.txt", "The Brothers Karmazov", "Fyodor Dostoevsky",'1880')
cp = make_it_df("/Users/andrewargaez/Author_Classifier/CP.txt", "Crime and Punishment", "Fyodor Dostoevsky",'1866')

gg = make_it_df("/Users/andrewargaez/Author_Classifier/GG.txt", "The Great Gatsby", "F. Scott Fitzgerald",'1925')
sop = make_it_df("/Users/andrewargaez/Author_Classifier/sop.txt", "This Side of Paradise", "F. Scott Fitzgerald",'1920')

pp = make_it_df("/Users/andrewargaez/Author_Classifier/PP.txt", "Pride and Prejudice", "Jane Austen",'1813')
em = make_it_df("/Users/andrewargaez/Author_Classifier/emma.txt", "Emma", "Jane Austen",'1815')

al= make_it_df("/Users/andrewargaez/Author_Classifier/alice.txt", "Alice in Woderland", "Lewis Carrol","1865")
tlg= make_it_df("/Users/andrewargaez/Author_Classifier/tlg.txt", "Through the Looking Glass", "Lewis Carrol","1871")

drac= make_it_df("/Users/andrewargaez/Author_Classifier/drac.txt", "Dracula", "Bram Stoker","1897")
lf= make_it_df("/Users/andrewargaez/Author_Classifier/LF.txt", "Lord of the Flies", "William Golding","1954")
pet = make_it_df("/Users/andrewargaez/Author_Classifier/Peter.txt", "Peter Pan", "J. M. Barrie","1911")

pdg = make_it_df("/Users/andrewargaez/Author_Classifier/pdg.txt", "Picture of Dorian Gray", "Oscar Wilde","1890")

ti  = make_it_df("/Users/andrewargaez/Author_Classifier/TI.txt", "Treasure Island", "Robert Louis Stevenson","1882")

huck = make_it_df("/Users/andrewargaez/Author_Classifier/huck.txt", "Adventures of Huckleberry Fin", "Mark Twain","1884")
ts = make_it_df("/Users/andrewargaez/Author_Classifier/ts.txt", "Adventures of Tom Sawyer", "Mark Twain","1876")

wow = make_it_df("/Users/andrewargaez/Author_Classifier/wow.txt", "The War of the Worlds", "H. G. Wells","1897")
im = make_it_df("/Users/andrewargaez/Author_Classifier/IM.txt", "The Invisible Man", "H. G. Wells","1897")


In [193]:
new_df =pd.concat([bell,kar,gg,pp,al,huck,wow],ignore_index=True)
test_df= pd.concat([fta,cp,sop,em,tlg,ts,im],ignore_index=True)

In [194]:
def squeaky_clean(new_df):
    others = '“,”,’,—,_,.,——'
    vals = list(new_df['txt'].values)
    arr=[]

    for _sent in vals:
        sent=[]
        for word in _sent:
            for char in others:
                word = word.replace(char,'')
            sent.append(word)      
        arr.append(str(sent))
    return pd.Series(arr)

In [195]:
test_df['txt']= squeaky_clean(test_df)

In [196]:
test_df

,txt,title,author,year
0,"['late', 'summer', 'year', 'lived', 'house', '...",A Farewell to Arms,Ernest Hemingway,1929
1,"['snow', 'slanted', 'across', 'wind', 'bare', ...",A Farewell to Arms,Ernest Hemingway,1929
2,"['shook', 'faith', 'filthy', 'vile', 'book', '...",A Farewell to Arms,Ernest Hemingway,1929
3,"['candle-light', 'made', 'shadows', 'wall', 's...",A Farewell to Arms,Ernest Hemingway,1929
4,"['window', 'open', 'bed', 'made', 'blankets', ...",A Farewell to Arms,Ernest Hemingway,1929
...,...,...,...,...
1227,"['wife', 'however', 'subsequently', 'reminded'...",The Invisible Man,H. G. Wells,1897
1228,"['passed', 'staircase', 'window', 'heard', 'si...",The Invisible Man,H. G. Wells,1897
1229,"['spread', 'cried', 'one', 'kemp', 'suddenly',...",The Invisible Man,H. G. Wells,1897
1230,"['mouth', 'wet', 'said', 'good', 'god', 'stood...",The Invisible Man,H. G. Wells,1897


In [197]:
def get_data(item):
    data=item.txt
    labels = item.author
    le = LabelEncoder()
    y = le.fit_transform(labels)
    return np.array(data), np.array(y)

In [198]:
def tune_naive_bayes(data, y):
    print("tuning naive bayes...")
    kfold = KFold(5)
    alphas = np.concatenate((np.arange(0, 0.1, 0.02), np.arange(.1, 1.3, 0.1)))
    scores = defaultdict(list)
    for train_index, test_index in kfold.split(data):
        data_train, data_test = data[train_index], data[test_index]
        y_train, y_test = y[train_index], y[test_index]
        tfidf = TfidfVectorizer()
        X_train = tfidf.fit_transform(data_train)
        print(X_train.shape)
        X_test = tfidf.transform(data_test)
        for alpha in alphas:
            nb = MultinomialNB(alpha=alpha)
            nb.fit(X_train, y_train)
            scores[alpha].append(nb.score(X_test, y_test))

    print("alpha  score")
    for alpha in alphas:
        print(" %.2f  %f" % (alpha, np.average(scores[alpha])))

In [199]:
def run_models(data, y):
    data_train, data_test, y_train, y_test = train_test_split(data, y)
    

    tfidf = TfidfVectorizer()
    X_train = tfidf.fit_transform(data_train).toarray()
    X_test = tfidf.transform(data_test).toarray()
    v_train = tfidf.fit_transform(test1).toarray()
    v_test = tfidf.transform(y1_test).toarray()
    

    models = [("Random Forest", RandomForestClassifier()),
              ("Decision Tree", DecisionTreeClassifier()),
              ("kNN", KNeighborsClassifier()),  
              ("Naive Bayes", MultinomialNB()),
              #("SVM", OneVsRestClassifier(SVC())),
              ("Logistic", OneVsRestClassifier(LogisticRegression()))]

    print("%20s %7s %9s %9s" % ("Name", "Score", "TrainTime", "TestTime"))

    for name, model in models:
        start = time.time()
        model.fit(X_train, y_train)
        trained = time.time()
        score = model.score(X_train, y_test)
        tested = time.time()
        

        print("%20s   %.3f %9s %9s" % (name, score,
                                       str(round(trained - start, 2)),
                                       str(round(tested - trained, 2))))

In [201]:
data1, y1 = get_data(test_df)
tune_naive_bayes(data1, y1)
run_models(data, y)

tuning naive bayes...
(985, 23165)


/Users/andrewargaez/opt/anaconda3/lib/python3.8/site-packages/sklearn/naive_bayes.py:508: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn('alpha too small will result in numeric errors, '


(985, 23209)


/Users/andrewargaez/opt/anaconda3/lib/python3.8/site-packages/sklearn/naive_bayes.py:508: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn('alpha too small will result in numeric errors, '


(986, 21142)


...

(986, 20100)


/Users/andrewargaez/opt/anaconda3/lib/python3.8/site-packages/sklearn/naive_bayes.py:508: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn('alpha too small will result in numeric errors, '


(986, 20461)
alpha  score
 0.00  0.406494
 0.02  0.452006
 0.04  0.427613
 0.06  0.384523
 0.08  0.364198
 0.10  0.354442
 0.20  0.330055
 0.30  0.321118
 0.40  0.315444
 0.50  0.309776
 0.60  0.304105
 0.70  0.302485
 0.80  0.296817
 0.90  0.292769
 1.00  0.286291
 1.10  0.285481
 1.20  0.283862


/Users/andrewargaez/opt/anaconda3/lib/python3.8/site-packages/sklearn/naive_bayes.py:508: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn('alpha too small will result in numeric errors, '


AttributeError: 'list' object has no attribute 'lower'

In [ ]:
tfidf = TfidfVectorizer()
v_train = tfidf.fit_transform(data1).toarray()
v_test = tfidf.transform(y1).toarray()

AttributeError: 'numpy.int64' object has no attribute 'lower'

In [ ]:
data, y = get_data(new_df)
tune_naive_bayes(data, y)
run_models(data, y)

tuning naive bayes...


AttributeError: 'list' object has no attribute 'lower'